<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Линейная регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка


Для начала загрузим переданные нам данные о комментариях с магазина "Викишоп" с разметкой об их токсичности.


In [1]:
#импорт необходимых библиотек
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re
from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0



В данных два столбца:

* <i> text </i> - непосредственно текст комментария

* <i> toxic </i> - индикатор его токсичности (0 - позитивный, 1 - негативный).


In [3]:
data.isna().sum()

text     0
toxic    0
dtype: int64


Видим, что данным требуется предобработка - комментарии написаны в разном регистре, также необходимо удалить знаки препинания.


In [4]:
data['text'] = data['text'].str.lower()
data.head()

,text,toxic
0,explanation\nwhy the edits made under my usern...,0
1,d'aww! he matches this background colour i'm s...,0
2,"hey man, i'm really not trying to edit war. it...",0
3,"""\nmore\ni can't make any real suggestions on ...",0
4,"you, sir, are my hero. any chance you remember...",0


In [5]:
#функция для очистки текста от знаков препинания
def clean_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return ' '.join(text.split())

In [6]:
data['text'] = data.text.apply(clean_text)
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0



Проведем лемматизацию текста комментариев.


In [7]:
token = nltk.tokenize.WhitespaceTokenizer()
lemm = nltk.stem.WordNetLemmatizer()

In [8]:
def lemm_text(text):
    return [lemm.lemmatize(w) for w in token.tokenize(text)]

In [9]:
data['text_lemm'] = data.text.apply(lemm_text)


Теперь, имея первоначальный вид слов, используемых в комментариях, можно приступить к этапу обучения моделей.


## Обучение


Создадим мешок слов и избавимся от стоп-слов.


In [10]:
corpus = data['text_lemm'].astype('U').values

In [11]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Теперь мы можем рассчитать оценки качества слов в мешке и разделить данные на признаки и выборки: обучающую, валидационную и тестовую.


In [12]:
#оценка качества слов
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
#определяем признаки
features = corpus
#целевой признак
target = data['toxic']
#делим признаки на выбоки
features_data, features_test, target_data, target_test = (
    train_test_split(features, target, test_size=0.2, random_state=42)
)
features_train, features_valid, target_train, target_valid = (
    train_test_split(features_data, target_data, test_size=0.25, random_state=42)
)
features_train = count_tf_idf.fit_transform(features_train)
features_valid = count_tf_idf.transform(features_valid)
features_test = count_tf_idf.transform(features_test)


Теперь перед нами стоит задача классификации: нужно обучить модели, которые бы предсказывали токсичный ли комментарий.



Обучим три модели: линейной регрессии, дерева решений и случайного леса. Лучшую модель определим по метрике F1.


### Линейная регрессия

<a id='correct3'></a>

In [13]:
model_linear = LogisticRegression(random_state=0, C=5)
model_linear.fit(features_train, target_train)
predictions = model_linear.predict(features_test)
result = f1_score(target_test, predictions)
print('F1 на модели линейной регрессии', result)

F1 на модели линейной регрессии 0.768904593639576


### Дерево решений

In [14]:
#зададим переменные, в которые сохраним наилучшие результаты
best_depth_tree = 0
best_result_tree = 0
best_model_tree = None
#в цикле перебираем значения глубины дерева
for depth in range(1, 11):
    #зададим модель с текущей глубиной
    model = DecisionTreeClassifier(random_state=0, max_depth=depth)
    #обучаем модель на обучающей выборке
    model.fit(features_train, target_train)
    #предсказываем результаты на валидационной выборке
    predictions = model.predict(features_valid)
    #рассчитываем RMSE
    result = f1_score(target_valid, predictions)
    #выбираем наилучший результат
    if result > best_result_tree:
        best_model_tree = model
        best_result_tree = result
        best_depth_tree = depth
print('Лучший F1 модели дерева решений:', best_result_tree)

Лучший F1 модели дерева решений: 0.5986478180700676


### Случайный лес

In [15]:
#зададим переменные, в которые сохраним наилучшие результаты
best_depth_forest = 0
best_est_forest = 0
best_result_forest = 0
best_model_forest = None
#в цикле перебираем значения глубины дерева и количества деревьев
for est in range(10, 51, 5):
    for depth in range(1, 11):
        #зададим модель
        model = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth)
        #обучаем модель на обучающей выборке
        model.fit(features_train, target_train)
        #предсказываем результаты на валидационной выборке
        predictions = model.predict(features_valid)
        #рассчитываем RMSE
        result = f1_score(target_valid, predictions)
        #выбираем наилучший результат
        if result > best_result_forest:
            best_model_forest = model
            best_result_forest = result
            best_depth_forest = depth
            best_est_forest = est
print('Лучший F1 модели случайного леса:', best_result_forest)

Лучший F1 модели случайного леса: 0


## Выводы


В данной работе мы выполнили проект по анализу комментариев интернет-магазина. Чтобы выделить среди отзывов, оставляемых покупателями, токсичные, мы создали три модели, обучаемые на основных словах из текстов комментариев.

При анализе результатов моделей линейной регрессии, дерева решений и случайного леса, мы опирались на классификационную метрику F1. Согласно ей, наилучший результат показала модель случайного леса и в дальнейшем мы рекомендуем использовать ее для классификации комментариев интернет-магазина.
